# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 6 2023 10:15AM,254451,19,8997614,"Graystone, LLC",Released
1,Jan 6 2023 10:12AM,254450,10,Enova-11172,Emerginnova,Released
2,Jan 6 2023 10:09AM,254449,19,8997623,"Graystone, LLC",Released
3,Jan 6 2023 10:07AM,254448,10,0086284769,ISDIN Corporation,Released
4,Jan 6 2023 10:07AM,254448,10,0086284775,ISDIN Corporation,Released
5,Jan 6 2023 10:07AM,254448,10,0086284774,ISDIN Corporation,Released
6,Jan 6 2023 10:07AM,254448,10,0086284776,ISDIN Corporation,Released
7,Jan 6 2023 9:56AM,254446,19,PBIO-103655,Paragon BioTeck,Executing
8,Jan 6 2023 9:56AM,254446,19,PBIO-103756,Paragon BioTeck,Released
9,Jan 6 2023 9:54AM,254445,10,CTF0011574,"Citieffe, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
52,254446,Released,1
53,254448,Released,4
54,254449,Released,1
55,254450,Released,1
56,254451,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254446,NaN,1.0,1.0
254448,NaN,NaN,4.0
254449,NaN,NaN,1.0
254450,NaN,NaN,1.0
254451,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254331,0.0,0.0,1.0
254346,0.0,0.0,1.0
254347,0.0,0.0,1.0
254351,0.0,0.0,2.0
254357,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254331,0,0,1
254346,0,0,1
254347,0,0,1
254351,0,0,2
254357,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254331,0,0,1
1,254346,0,0,1
2,254347,0,0,1
3,254351,0,0,2
4,254357,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254331,,,1
1,254346,,,1
2,254347,,,1
3,254351,,,2
4,254357,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 6 2023 10:15AM,254451,19,"Graystone, LLC"
1,Jan 6 2023 10:12AM,254450,10,Emerginnova
2,Jan 6 2023 10:09AM,254449,19,"Graystone, LLC"
3,Jan 6 2023 10:07AM,254448,10,ISDIN Corporation
7,Jan 6 2023 9:56AM,254446,19,Paragon BioTeck
9,Jan 6 2023 9:54AM,254445,10,"Citieffe, Inc."
10,Jan 6 2023 9:48AM,254444,10,Eywa Pharma Inc.
16,Jan 6 2023 9:12AM,254439,10,"Citieffe, Inc."
18,Jan 6 2023 9:05AM,254440,10,Bio-PRF
19,Jan 6 2023 8:54AM,254436,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 6 2023 10:15AM,254451,19,"Graystone, LLC",,,1
1,Jan 6 2023 10:12AM,254450,10,Emerginnova,,,1
2,Jan 6 2023 10:09AM,254449,19,"Graystone, LLC",,,1
3,Jan 6 2023 10:07AM,254448,10,ISDIN Corporation,,,4
4,Jan 6 2023 9:56AM,254446,19,Paragon BioTeck,,1,1
5,Jan 6 2023 9:54AM,254445,10,"Citieffe, Inc.",,,1
6,Jan 6 2023 9:48AM,254444,10,Eywa Pharma Inc.,,,6
7,Jan 6 2023 9:12AM,254439,10,"Citieffe, Inc.",,,2
8,Jan 6 2023 9:05AM,254440,10,Bio-PRF,,,1
9,Jan 6 2023 8:54AM,254436,10,ISDIN Corporation,,,60


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 10:15AM,254451,19,"Graystone, LLC",1,,
1,Jan 6 2023 10:12AM,254450,10,Emerginnova,1,,
2,Jan 6 2023 10:09AM,254449,19,"Graystone, LLC",1,,
3,Jan 6 2023 10:07AM,254448,10,ISDIN Corporation,4,,
4,Jan 6 2023 9:56AM,254446,19,Paragon BioTeck,1,1,
5,Jan 6 2023 9:54AM,254445,10,"Citieffe, Inc.",1,,
6,Jan 6 2023 9:48AM,254444,10,Eywa Pharma Inc.,6,,
7,Jan 6 2023 9:12AM,254439,10,"Citieffe, Inc.",2,,
8,Jan 6 2023 9:05AM,254440,10,Bio-PRF,1,,
9,Jan 6 2023 8:54AM,254436,10,ISDIN Corporation,60,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 10:15AM,254451,19,"Graystone, LLC",1,,
1,Jan 6 2023 10:12AM,254450,10,Emerginnova,1,,
2,Jan 6 2023 10:09AM,254449,19,"Graystone, LLC",1,,
3,Jan 6 2023 10:07AM,254448,10,ISDIN Corporation,4,,
4,Jan 6 2023 9:56AM,254446,19,Paragon BioTeck,1,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 10:15AM,254451,19,"Graystone, LLC",1.0,NaN,NaN
1,Jan 6 2023 10:12AM,254450,10,Emerginnova,1.0,NaN,NaN
2,Jan 6 2023 10:09AM,254449,19,"Graystone, LLC",1.0,NaN,NaN
3,Jan 6 2023 10:07AM,254448,10,ISDIN Corporation,4.0,NaN,NaN
4,Jan 6 2023 9:56AM,254446,19,Paragon BioTeck,1.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 10:15AM,254451,19,"Graystone, LLC",1.0,0.0,0.0
1,Jan 6 2023 10:12AM,254450,10,Emerginnova,1.0,0.0,0.0
2,Jan 6 2023 10:09AM,254449,19,"Graystone, LLC",1.0,0.0,0.0
3,Jan 6 2023 10:07AM,254448,10,ISDIN Corporation,4.0,0.0,0.0
4,Jan 6 2023 9:56AM,254446,19,Paragon BioTeck,1.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6105867,351.0,12.0,11.0
15,508757,5.0,0.0,0.0
16,1272050,3.0,2.0,0.0
18,254420,1.0,0.0,0.0
19,3561794,26.0,3.0,4.0
20,763182,3.0,0.0,0.0
21,763091,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6105867,351.0,12.0,11.0
1,15,508757,5.0,0.0,0.0
2,16,1272050,3.0,2.0,0.0
3,18,254420,1.0,0.0,0.0
4,19,3561794,26.0,3.0,4.0
5,20,763182,3.0,0.0,0.0
6,21,763091,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,351.0,12.0,11.0
1,15,5.0,0.0,0.0
2,16,3.0,2.0,0.0
3,18,1.0,0.0,0.0
4,19,26.0,3.0,4.0
5,20,3.0,0.0,0.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,351.0
1,15,Released,5.0
2,16,Released,3.0
3,18,Released,1.0
4,19,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,11.0,0.0,0.0,0.0,4.0,0.0,0.0
Executing,12.0,0.0,2.0,0.0,3.0,0.0,0.0
Released,351.0,5.0,3.0,1.0,26.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,11.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,12.0,0.0,2.0,0.0,3.0,0.0,0.0
2,Released,351.0,5.0,3.0,1.0,26.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,11.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,12.0,0.0,2.0,0.0,3.0,0.0,0.0
2,Released,351.0,5.0,3.0,1.0,26.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()